In [ ]:
import numpy as np
import pandas as pd
import pandas_profiling as pdp
import os
import glob
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold, cross_val_score, cross_val_predict
from sklearn.metrics import log_loss, make_scorer
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
pd.set_option('display.max_columns', None)

In [ ]:
data_dir = '../input/mens-march-mania-2022/MDataFiles_Stage1'
dirs = glob.glob(os.path.join(data_dir, '*.csv'))
for dir in dirs:
    var_name = os.path.splitext(os.path.basename(dir))[0]
    print("{} start loading".format(var_name))
    globals()[var_name] = pd.read_csv(dir, encoding="cp932")

## NCAA Division I Men's Basketball Tournamentとは？

NCAA Division I Men's Basketball Tournamentとは，アメリカ合衆国における大学のバスケットボール全国大会である．日本で言うインターカレッジ(インターハイの大学版)みたいなもの．NCAA March Madnessとも呼ばれている．アメリカではこの大会は大変人気なようで，大会の結果予想は一種の文化となっている．

基本的な情報は[日本語版wikipedia](https://ja.wikipedia.org/wiki/NCAA%E7%94%B7%E5%AD%90%E3%83%90%E3%82%B9%E3%82%B1%E3%83%83%E3%83%88%E3%83%9C%E3%83%BC%E3%83%AB%E3%83%88%E3%83%BC%E3%83%8A%E3%83%A1%E3%83%B3%E3%83%88)にも記載されているが，より詳しい情報を知るには[英語版wikipedia](https://en.wikipedia.org/wiki/NCAA_Division_I_Men%27s_Basketball_Tournament)を参照する必要があるだろう．

現在の方式の場合，参加チームは68チームで，32チームは32あるカンファレンスの優勝チームから選ばれ，残りは選定委員会によって選ばれる．kaggleによって提供されるデータは，最終的な全国大会のデータの他にも，Division Iに属するチームの多数の試合データが含まれている．

ここでは，この大会に詳しくない人たち(自分もその一人)に向けて，wikipediaに記載されている情報を交えて，kaggleから提供されたデータを眺めてみる．なお，本コンペティションでは，kaggleが提供するデータ以外のデータも使用することが推奨されている．

Notebookの投稿はこれが初めてなので，至らないところがあれば教えてほしい．

I suppose the main readers of this article will be Japanese speakers, but I'm planning to add English descriptions.

### MTeams.csv

書く大学には，4桁の数字(TeamID)が割り当てられている．全てのチームが毎年参加しているわけではなく，初めて参加した年(FirstD1Season)と最後に参加した年(LastD1Season)が記載されている．大学数は372．

In [ ]:
MTeams

### MSeasons.csv

現在の大会方式では68チームが出場する．チーム数は開催年によって異なる．詳細は次のセクションで述べる．

各チームは，委員会によって4つのリージョンに割り当てられる．リージョンは地域に基づいて決められている．例えば，1985年では，East, West, MidWest, Southeastの4つのリージョンがあったようだ．ただし，なるべくリージョン間でチームの質の差が出ないように委員会はリージョンを定めている．各リージョンの名称は年度によって異なるので，アルファベット順で最初のリージョンがRegionW，準決勝でRegionWと対戦したチームがRegionX，残りのリージョンのうちアルファベット順で最初のチームがRegionYとなる．したがって，残るRegionZは，準決勝でRegionYと対戦しているはずである．

また，DayZeroはその大会の基準日となる日で，後述のMRegularSeasonCompactResultsなどでは，ゲームの開催日がDayZeroから何日後か(DayNum)が記載されている．

In [ ]:
MSeasons.head()

### MNCAATourneySeeds.csv

このファイルには，各年度の大会におけるSeedが記載されている．日本語でシードというと，最初の試合を免除されているチームを想像してしまうが，ここでのSeedはそれとは意味合いが異なる．Seedは各リージョンにおけるチームの強さのランキングのようなもので，委員会が決定している．なお，全国大会(Tourney)に出場するチームにのみSeedが割り当てられるため，後述のRegular Seasonの出場チームの中にはこのSeedが割り当てられていないチームも存在する．また，2022年の全国大会はまだ始まっていないので，2022年のデータは存在しない(2022年のRegular Seasonは既に開催されている)．

具体的にデータを見ながら確認してみよう．最初の行を見てみると，1985年，TeamID:1207のチームは，RegionWでW01というSeedが割り当てられていることが分かる．これは，そのチームがRegionWで最も強いチームだと委員会にランク付けされていることを意味する．1985年では，64チームが4つのリージョンに等分配されているので，W16のSeedが割り当てられたチームはRegionWで最も弱いと目されているチームとなる．

ゲームはいくつかのラウンドに分かれており，The First Roundでは，W01とW16，W02とW15．．．という組み合わせで試合を行う．このように，強いチームと弱いチームを戦わせることによって，強いチームが早い段階で他の強いチームに負けて敗退することを防いでいる．この大会は勝ち抜き戦(一度負けたら敗退)なので，The First Roundで64チームのうち，32チームのみが次のThe Second Roundに進むことになる．The Second Roundも同様の方式で開催され，32チームのうち16チームのみが次のラウンド(The Regional Semi-finals, リージョンでの準決勝)に進むことになる．

なお，2001\~2010年は出場チームが65チーム，2011年以降は68チームに出場数が増えている．2001年の場合では，RegionYが17チームになっており，SeedにY16a，Y16bが存在している．2001\~2010年では，The First Roundに先立ち，このY16aとY16bがplay-in gameを行い，負けたチームを除いた64チームでその後のThe First Roundを行った．また，2011年以降は68チームの場合には，各リージョンにそれぞれ17チームが所属し，下位のSeedのチーム同士(例えばW16aとW16b)でThe First Four呼ばれるゲームを行い，負けた4チームを除いた64チームがThe First Roundへ参加する．

In [ ]:
MNCAATourneySeeds['SeedNum'] = MNCAATourneySeeds['Seed'].str.extract('(\d+)').astype(int)
MNCAATourneySeeds

In [ ]:
MNCAATourneySeeds_groups = MNCAATourneySeeds.groupby('Season', as_index=False).count().set_index(['Season', 'Seed']).drop('TeamID',axis=1).groupby(level='Seed').groups
for i in MNCAATourneySeeds_groups.keys():
    print("year: {}-{}, the number of teams: {}".format(min(MNCAATourneySeeds_groups[i])[0], max(MNCAATourneySeeds_groups[i])[0], i))

In [ ]:
MNCAATourneySeeds

In [ ]:
MNCAATourneySeeds.query('Seed.str.startswith("Y")', engine='python').query('Season==2001')

### MRegularSeasonCompactResults.csv

このファイルでは，Regular Season(全国大会前の期間)のゲームの詳細が記載されている．DayNumには，シーズン開始から何日後かが記されていて，Regular Seasonのみを含む今回のデータでは132が最大値となっている．なお，2022年(本年度)のデータも含まれているが，もちろんMNCAATourneyCompactResultsの2022の結果はまだ出ていないので，MNCAATourneyCompactResultsには記載されていない．

WTeamIDは勝ったチームのID，WTeamIDは負けたチームのIDを表す．また，WScoreは勝ったチームが，LScoreは負けたチームが試合中に獲得した点数である．当然，WScore>LScoreの関係が成立する．

WLocは試合開催地の種類を示す．Hは勝者チームのホームタウンであることを示し，Aは敗者のホームタウンであったことを示す．Nはどちらもない中立の場合を表す．中立であるかどうかという判定は，Kenneth Massey dataに基づいているらしい．

NumOTは延長戦が行われた回数を示す．バスケットボールでは，第4ピリオドで勝敗が消しなかった場合は5分間の延長戦を決着がつくまで繰り返す．

In [ ]:
MRegularSeasonCompactResults['ScoreDiff'] = MRegularSeasonCompactResults['WScore']-MRegularSeasonCompactResults['LScore']
MRegularSeasonCompactResults

In [ ]:
MRegularSeasonCompactResults.describe()

#### ゲームの開催場所と試合でついて点差の関係

勝つ場合，ホームタウンでの勝利だった場合が圧倒的に多い．中立の場所ではそもそも試合数が少ないようだ．

In [ ]:
x = 4
y = int(36/x)
plt.rcParams["font.size"] = 32
fig = plt.figure(figsize=(x*10, y*10))
for i, season in enumerate(range(1985, 1985+x*y)):
    ax = fig.add_subplot(y, x, i+1)
    sns.countplot(data=MRegularSeasonCompactResults[MRegularSeasonCompactResults['Season']==season], 
                 x='WLoc', order=['H', 'N', 'A'], ax=ax)
    ax.set_title(season)
fig.tight_layout()


大差で試合が終了するということは少ないことがわかる．

In [ ]:
x = 4
y = int(36/x)
plt.rcParams["font.size"] = 32
fig = plt.figure(figsize=(x*10, y*10))
for i, season in enumerate(range(1985, 1985+x*y)):
    ax = fig.add_subplot(y, x, i+1)
    sns.histplot(data=MRegularSeasonCompactResults[MRegularSeasonCompactResults['Season']==season], 
                 x='ScoreDiff', hue='WLoc', bins=10, hue_order=['H', 'N', 'A'], ax=ax)
    ax.set_title(season)
    plt.xlim(0, 100)
fig.tight_layout()

ホームタウンで行われた試合の方が，中立の場所や相手のホームタウンで行われた試合よりも点差が大きくつくことが多いことがわかる．

In [ ]:
x = 4
y = int(36/x)
plt.rcParams["font.size"] = 32
fig = plt.figure(figsize=(x*10, y*10))
for i, season in enumerate(range(1985, 1985+x*y)):
    ax = fig.add_subplot(y, x, i+1)
    sns.boxplot(data=MRegularSeasonCompactResults[MRegularSeasonCompactResults['Season']==season], 
                 x='WLoc', y='ScoreDiff', order=['H', 'N', 'A'], ax=ax)
    sns.stripplot(data=MRegularSeasonCompactResults[MRegularSeasonCompactResults['Season']==season], 
                 x='WLoc', y='ScoreDiff', order=['H', 'N', 'A'], ax=ax, color='.3')
    
    ax.set_title(season)
#     plt.xlim(0, 100)
fig.tight_layout()

### 

### MNCAATourneyCompactResults.csv

先述のMRegularSeasonCompactResultsの全国大会版．DayNumによってどのラウンドのゲームなのかがわかる．詳細は以下．

以下の試合以外にも，全国大会以外の大会(Secondary Tourney)のデータが含まれている．また，2020年は試合が行われなかったので注意．

ちなみに，基本的に試合はホームコートで行われないようになっている．ただし，準決勝戦・決勝戦の開催場所は大会開催前に既に決まっているので，偶然ホームコートで試合が行われる可能性もある．今回のデータでは，ゲームの開催場所(WLoc)は全て中立(N)になっている．

| DayNum  | Rounds                    | the number of team(before) | the number of team(after) | 備考                                                            |
|---------|---------------------------|----------------------------|---------------------------|-----------------------------------------------------------------|
| 134,135 | play-in games(Round0)             | 65以上(年によって異なる)   | 64                        | 65チーム以上ある場合に，The First Roundに先立って行われるゲーム |
| 136,137 | The First Round(Round1)           | 64                         | 32                        | 地区ごとに行われる．                                            |
| 138,139 | The Second Round(Round2)          | 32                         | 16                        | 地区ごとに行われる．                                            |
| 143,144 | The Regional Semi-finals(Round3)  | 16                         | 8                         | 地区準決勝                                                      |
| 145,146 | The Regional Finals(Round4)       | 8                          | 4                         | 地区決勝                                                        |
| 152     | The National Semi-finals(Round5)  | 4                          | 2                         | 全国大会準決勝                                                  |
| 154     | The National Championship(Round6) | 2                          | 1                         | 全国大会決勝                                                    |

In [ ]:
# WLocにはNしか含まれていない
MNCAATourneyCompactResults['WLoc'].unique()

In [ ]:
# 上述の表以外の日数も含まれているので注意
sorted(MNCAATourneyCompactResults['DayNum'].unique())

In [ ]:
DayNums = [134, 135, 136, 137, 138, 139, 140, 143, 144, 145, 146, 147, 148, 152, 154]
Rounds = [0, 0, 1, 1, 2, 2, np.nan, 3, 3, 4, 4, np.nan, np.nan, 5, 6]
for d, r in zip(DayNums, Rounds):
    MNCAATourneyCompactResults.loc[MNCAATourneyCompactResults['DayNum']==d,'Round'] = r
MNCAATourneyCompactResults['ScoreDiff'] = MNCAATourneyCompactResults['WScore'] - MNCAATourneyCompactResults['LScore']
MNCAATourneyCompactResults

In [ ]:
MNCAATourneyCompactResults.describe()

#### 

#### 得点差の傾向

MRegularSeasonCompactResultsと同じく，ほとんどの試合が僅差で終わることがわかる．

In [ ]:
x = 4
y = int(36/x)
plt.rcParams["font.size"] = 32
fig = plt.figure(figsize=(x*10, y*10))
for i, season in enumerate(range(1985, 1985+x*y)):
    ax = fig.add_subplot(y, x, i+1)
    sns.histplot(data=MNCAATourneyCompactResults[MNCAATourneyCompactResults['Season']==season], 
                 x='ScoreDiff', ax=ax)
    ax.set_title(season)
    plt.xlim(0, 60)
fig.tight_layout()

#### Roundと点数差の関係

この大会では，特にRound1で，強いチームと弱いチームが当たることになる．大会のシステムを考えると，Roundが進むにつれてチームの実力差も縮んでいくことだろう．従って，後半のRoundであればあるほど点数差が小さいことが予想される．実際，概ねそのような傾向が見てとれる．

In [ ]:
x = 4
y = int(36/x)
plt.rcParams["font.size"] = 32
fig = plt.figure(figsize=(x*10, y*10))
for i, season in enumerate(range(1985, 1985+x*y)):
    if season==2020:
        season=2021
    ax = fig.add_subplot(y, x, i+1)
    sns.boxplot(data=MNCAATourneyCompactResults[MNCAATourneyCompactResults['Season']==season].query('Round<7'), 
                 x='Round', y='ScoreDiff', ax=ax)
    sns.stripplot(data=MNCAATourneyCompactResults[MNCAATourneyCompactResults['Season']==season].query('Round<7'),  
                 x='Round', y='ScoreDiff', ax=ax, color='.3')
    
    ax.set_title(season)
#     plt.xlim(0, 100)
fig.tight_layout()

### ここまでのデータの結合



In [ ]:
MNCAATourneyCompactResults['DayNum'].sort_values()

In [ ]:
#　便宜的に，全国大会前の試合のRoundは全て-1にしておく．
MRegularSeasonCompactResults.loc[:, 'Round']=-1
# MNCAAResults = pd.concat([MNCAATourneyCompactResults.dropna(axis=0), MRegularSeasonCompactResults], axis=0).sort_values(['Season', 'DayNum'], axis=0).reset_index()
MNCAAResults = pd.concat([MRegularSeasonCompactResults, MNCAATourneyCompactResults], axis=0).reset_index(drop=True)
MNCAAResults = MNCAAResults.merge(MNCAATourneySeeds.rename(columns={'TeamID': 'WTeamID'}), how='left').rename(columns={'Seed':'WSeed', 'SeedNum': 'WSeedNum'})
MNCAAResults = MNCAAResults.merge(MNCAATourneySeeds.rename(columns={'TeamID': 'LTeamID'}), how='left').rename(columns={'Seed':'LSeed', 'SeedNum': 'LSeedNum'})
MNCAAResults = MNCAAResults.merge(MTeams.rename(columns={'TeamID': 'WTeamID'})).rename(columns={'TeamName':'WTeamName','FirstD1Season':'WFirstD1Season', 'LastD1Season':'WLastD1Season'})
MNCAAResults = MNCAAResults.merge(MTeams.rename(columns={'TeamID': 'LTeamID'})).rename(columns={'TeamName':'LTeamName','FirstD1Season':'LFirstD1Season', 'LastD1Season':'LLastD1Season'})
MNCAAResults['LLoc'] = MNCAAResults['WLoc'].map({'H':'A', 'N':'N', 'A':'H'})
MNCAAResults = MNCAAResults.reindex(columns=['Season', 'DayNum','Round',  'NumOT', 'ScoreDiff', 
                                             'WTeamID', 'WScore', 'WLoc', 'WSeed','WSeedNum','WTeamName','WFirstD1Season','WLastD1Season',
                                             'LTeamID', 'LScore', 'LLoc', 'LSeed','LSeedNum','LTeamName','LFirstD1Season','LLastD1Season'])
MNCAAResults = MNCAAResults.sort_values(['Season', 'DayNum'])
MNCAAResults

In [ ]:
MNCAAResults.query('Round>=-0.5')

### 例年優勝するチームのSeedは？

Seedは強い順に割り当てられるので，Seed番号が小さいチームの方が優勝する可能性は高いと予想できる．以下のグラフを見ると，優勝チームのほとんどはSeedが1だったことがわかる．また，Seed9以上が優勝したことは1985年以降，一度もない．バスケットボールには運の要素が少なく，実力主義であることが伺える．

In [ ]:
plt.rcParams["font.size"] = 16
fig = plt.figure(figsize=(10, 5))
ax = sns.countplot(data=MNCAAResults.query('Round==6'), x='WSeedNum')
ax.set_title('Seeds assigned to champions')

### その年や前年度までの勝率と全国大会での結果の関係

In [ ]:
MRegularSeasonCompactResults.query('Season==2021&WTeamID==1276')

In [ ]:
# Todo: WLocによって場合わけ
MRegularSeasonCompactGameSum = pd.concat([MRegularSeasonCompactResults[['Season', 'WTeamID', 'LTeamID', 'DayNum']], MRegularSeasonCompactResults[['Season','WTeamID', 'LTeamID', 'DayNum']].rename(columns={'WTeamID':'LTeamID', 'LTeamID':'WTeamID'})]).groupby(['Season', 'WTeamID', 'LTeamID'], as_index=False).count()
MRegularSeasonCompactGameSum = MRegularSeasonCompactGameSum.rename(columns={'DayNum': 'GameSum'})
MRegularSeasonCompactWinSum = MRegularSeasonCompactResults[['Season', 'WTeamID', 'LTeamID', 'DayNum']].groupby(['Season', 'WTeamID', 'LTeamID'], as_index=False).count()
MRegularSeasonCompactWinSum = MRegularSeasonCompactWinSum.rename(columns={'DayNum': 'WinSum'})
MRegularSeasonCompactWinRate = MRegularSeasonCompactGameSum.merge(MRegularSeasonCompactWinSum)
MRegularSeasonCompactWinRate['WinRate'] = MRegularSeasonCompactWinRate['WinSum'] / MRegularSeasonCompactWinRate['GameSum']
MRegularSeasonCompactWinRate = MRegularSeasonCompactWinRate.rename(columns={'WTeamID': 'ATeamID', 'LTeamID':'BTeamID', 'WinSum': 'ARegularSeasonWinSum', 'WinRate': 'ARegularSeasonWinRate'})
MRegularSeasonCompactWinRateMissing = MRegularSeasonCompactWinRate.query('GameSum==ARegularSeasonWinSum').rename(columns={'ATeamID': 'BTeamID', 'BTeamID': 'ATeamID'})
MRegularSeasonCompactWinRateMissing['ARegularSeasonWinRate'] = 0.
MRegularSeasonCompactWinRateMissing['ARegularSeasonWinSum'] = 0
MRegularSeasonCompactWinRate = pd.concat([MRegularSeasonCompactWinRate, MRegularSeasonCompactWinRateMissing]).sort_values(['Season', 'ATeamID', 'BTeamID']).reset_index(drop=True)
MRegularSeasonCompactWinRate = MRegularSeasonCompactWinRate.rename(columns={'GameSum': 'RegularSeasonGameSum'})
MRegularSeasonCompactWinRateAll = MRegularSeasonCompactWinRate[['ATeamID', 'RegularSeasonGameSum', 'ARegularSeasonWinSum', 'Season']].groupby(['ATeamID', 'Season'], as_index=False).sum()
MRegularSeasonCompactWinRateAll = MRegularSeasonCompactWinRateAll.rename(columns={'RegularSeasonGameSum': 'ARegularSeasonGameSumAll', 'ARegularSeasonWinSum': 'ARegularSeasonWinSumAll'})
MRegularSeasonCompactWinRateAll['ARegularSeasonWinRateAll'] = MRegularSeasonCompactWinRateAll['ARegularSeasonWinSumAll'] / MRegularSeasonCompactWinRateAll['ARegularSeasonGameSumAll']
# MRegularSeasonCompactWinRate = MRegularSeasonCompactWinRate.merge(MRegularSeasonCompactWinRateAll, how='left')
# MRegularSeasonCompactWinRate = MRegularSeasonCompactWinRate.merge(MRegularSeasonCompactWinRateAll.rename(columns={'ATeamID': 'BTeamID', 'ARegularSeasonGameSumAll': 'BRegularSeasonGameSumAll', 'ARegularSeasonWinSumAll': 'BRegularSeasonWinSumAll', 
#                                                                                                                  'ARegularSeasonWinRateAll': 'BRegularSeasonWinRateAll'}), how='left', on=['Season', 'BTeamID'])
MRegularSeasonCompactWinRate

In [ ]:
MNCAATourneyCompactResultsDouble = pd.concat([MNCAATourneyCompactResults, MNCAATourneyCompactResults.rename(columns={'WTeamID':'LTeamID', 'WScore': 'LScore', 'LTeamID':'WTeamID', 'LScore':'WScore'})]).reset_index(drop=True)
MNCAATourneyCompactResultsDouble['ScoreDiff'] = MNCAATourneyCompactResultsDouble['WScore'] - MNCAATourneyCompactResultsDouble['LScore']
MNCAATourneyCompactResultsDouble.drop(['WLoc', 'NumOT', 'Round', 'WScore', 'LScore', 'DayNum'], axis=1, inplace=True)
MNCAATourneyCompactResultsDouble['AWin'] = (MNCAATourneyCompactResultsDouble['ScoreDiff'] > 0).astype(int)
MNCAATourneyCompactResultsDouble.rename(columns={'WTeamID': 'ATeamID', 'LTeamID': 'BTeamID'}, inplace=True)
# MRegularSeasonCompactWinRate = MRegularSeasonCompactWinRate.merge(MNCAATourneyCompactResultsDouble, on=['Season', 'ATeamID', 'BTeamID'])
MNCAATourneyCompactResultsDouble = MNCAATourneyCompactResultsDouble.merge(MRegularSeasonCompactWinRateAll, how='left')
MNCAATourneyCompactResultsDouble = MNCAATourneyCompactResultsDouble.merge(MRegularSeasonCompactWinRateAll.rename(columns={'ATeamID': 'BTeamID', 'ARegularSeasonGameSumAll': 'BRegularSeasonGameSumAll', 'ARegularSeasonWinSumAll': 'BRegularSeasonWinSumAll', 
                                                                                                                 'ARegularSeasonWinRateAll': 'BRegularSeasonWinRateAll'}), how='left', on=['Season', 'BTeamID'])
MNCAATourneyCompactResultsDouble = MNCAATourneyCompactResultsDouble.merge(MRegularSeasonCompactWinRate, how='left')
MNCAATourneyCompactResultsDouble.fillna(0, inplace=True)
MNCAATourneyCompactResultsDouble = MNCAATourneyCompactResultsDouble.merge(MNCAATourneySeeds.rename(columns={'TeamID':'ATeamID'}), how='left').rename(columns={'SeedNum':'ASeedNum'}).drop('Seed', axis=1)
MNCAATourneyCompactResultsDouble = MNCAATourneyCompactResultsDouble.merge(MNCAATourneySeeds.rename(columns={'TeamID':'BTeamID'}), how='left').rename(columns={'SeedNum':'BSeedNum'}).drop('Seed', axis=1)
MNCAATourneyCompactResultsDouble['SeedDiff']=MNCAATourneyCompactResultsDouble['ASeedNum']-MNCAATourneyCompactResultsDouble['BSeedNum']
MNCAATourneyCompactResultsDouble

In [ ]:
# # RegularSeasonの場所ごとにも集計
# for l in ['H', 'N', 'A']:
#     MRegularSeasonCompactGameSum = pd.concat([MRegularSeasonCompactResults.query('WLoc==@l')[['Season', 'WTeamID', 'LTeamID', 'DayNum']], MRegularSeasonCompactResults.query('WLoc==@l')[['Season','WTeamID', 'LTeamID', 'DayNum']].rename(columns={'WTeamID':'LTeamID', 'LTeamID':'WTeamID'})]).groupby(['Season', 'WTeamID', 'LTeamID'], as_index=False).count()
#     MRegularSeasonCompactGameSum = MRegularSeasonCompactGameSum.rename(columns={'DayNum': 'GameSum'+l})
#     MRegularSeasonCompactWinSum = MRegularSeasonCompactResults.query('WLoc==@l')[['Season', 'WTeamID', 'LTeamID', 'DayNum']].groupby(['Season', 'WTeamID', 'LTeamID'], as_index=False).count()
#     MRegularSeasonCompactWinSum = MRegularSeasonCompactWinSum.rename(columns={'DayNum': 'WinSum'+l})
#     MRegularSeasonCompactWinRate = MRegularSeasonCompactGameSum.merge(MRegularSeasonCompactWinSum)
#     MRegularSeasonCompactWinRate['WinRate'+l] = MRegularSeasonCompactWinRate['WinSum'+l] / MRegularSeasonCompactWinRate['GameSum'+l]
#     MRegularSeasonCompactWinRate = MRegularSeasonCompactWinRate.rename(columns={'WTeamID': 'ATeamID', 'LTeamID':'BTeamID', 'WinSum'+l: 'ARegularSeasonWinSum'+l, 'WinRate'+l: 'ARegularSeasonWinRate'+l})
#     MRegularSeasonCompactWinRateMissing = MRegularSeasonCompactWinRate.query('GameSum{}==ARegularSeasonWinSum{}'.format(l,l)).rename(columns={'ATeamID': 'BTeamID', 'BTeamID': 'ATeamID'})
#     MRegularSeasonCompactWinRateMissing['ARegularSeasonWinRate'+l] = 0.
#     MRegularSeasonCompactWinRateMissing['ARegularSeasonWinSum'+l] = 0
#     MRegularSeasonCompactWinRate = pd.concat([MRegularSeasonCompactWinRate, MRegularSeasonCompactWinRateMissing]).sort_values(['Season', 'ATeamID', 'BTeamID']).reset_index(drop=True)
#     MRegularSeasonCompactWinRate = MRegularSeasonCompactWinRate.rename(columns={'GameSum'+l: 'RegularSeasonGameSum'+l})
#     X = X.merge(MRegularSeasonCompactWinRate, how='left')
# X.fillna(0, inplace=True)
# X

## MSampleSubmissionStage1.csv

提出ファイルの形式を確認してみよう．IDは(年度)\_(TeamID(小さい方))\_(TeamID(大きい方))となっており，Predの列にTeamIDが小さい方が勝つ確率を記入する．

In [ ]:
MSampleSubmissionStage1

In [ ]:
MSampleSubmissionStage1['Season'] = MSampleSubmissionStage1['ID'].apply(lambda x: int(x.split('_')[0]))
MSampleSubmissionStage1['ATeamID'] = MSampleSubmissionStage1['ID'].apply(lambda x: int(x.split('_')[1]))
MSampleSubmissionStage1['BTeamID'] = MSampleSubmissionStage1['ID'].apply(lambda x: int(x.split('_')[2]))
MSampleSubmissionStage1Double = pd.concat([MSampleSubmissionStage1, MSampleSubmissionStage1.rename({'ATeamID': 'BTeamID', 'BTeamID':'ATeamID'}, axis=1)])
MSampleSubmissionStage1Double = MSampleSubmissionStage1Double.merge(MRegularSeasonCompactWinRate, how='left', on=['Season', 'ATeamID', 'BTeamID'])
MSampleSubmissionStage1Double.query('Season==2021&ATeamID==1101&BTeamID==1116')
MSampleSubmissionStage1Double.fillna(0, inplace=True)
MSampleSubmissionStage1Double = MSampleSubmissionStage1Double.merge(MRegularSeasonCompactWinRateAll, how='left')
MSampleSubmissionStage1Double = MSampleSubmissionStage1Double.merge(MRegularSeasonCompactWinRateAll.rename(columns={'ATeamID': 'BTeamID', 'ARegularSeasonGameSumAll': 'BRegularSeasonGameSumAll', 'ARegularSeasonWinSumAll': 'BRegularSeasonWinSumAll', 
                                                                                                                 'ARegularSeasonWinRateAll': 'BRegularSeasonWinRateAll'}), how='left', on=['Season', 'BTeamID'])
MSampleSubmissionStage1Double = MSampleSubmissionStage1Double.merge(MNCAATourneyCompactResultsDouble, how='left')
MSampleSubmissionStage1Double.drop(['ASeedNum', 'BSeedNum'], axis=1, inplace=True)
MSampleSubmissionStage1Double = MSampleSubmissionStage1Double.merge(MNCAATourneySeeds.rename(columns={'TeamID':'ATeamID'}), how='left').rename(columns={'SeedNum':'ASeedNum'}).drop('Seed', axis=1)
MSampleSubmissionStage1Double = MSampleSubmissionStage1Double.merge(MNCAATourneySeeds.rename(columns={'TeamID':'BTeamID'}), how='left').rename(columns={'SeedNum':'BSeedNum'}).drop('Seed', axis=1)
MSampleSubmissionStage1Double['SeedDiff']=MSampleSubmissionStage1Double['ASeedNum']-MSampleSubmissionStage1Double['BSeedNum']
MSampleSubmissionStage1Double

In [ ]:
MNCAATourneyCompactResultsDouble

In [ ]:
# X = MNCAATourneyCompactResultsDouble.merge(MSampleSubmissionStage1Double, how='inner').reset_index(drop=True)
# 全ての年度のデータを使用する場合
X = MNCAATourneyCompactResultsDouble
y_train = X['AWin']
# X_train = X.drop(['AWin', 'Pred', 'ID', 'ScoreDiff'], axis=1)
X_train = X

In [ ]:
features = ['SeedDiff', 'ARegularSeasonGameSumAll', 'ARegularSeasonWinSumAll','ARegularSeasonWinRateAll', 
           'BRegularSeasonGameSumAll', 'BRegularSeasonWinSumAll', 'BRegularSeasonWinRateAll',
           'RegularSeasonGameSum', 'ARegularSeasonWinSum', 'ARegularSeasonWinRate', 'ASeedNum', 'BSeedNum']
seasons = [2016, 2017, 2018, 2019, 2021]
models = [LogisticRegression(random_state=0, solver='liblinear'),
         LGBMClassifier(random_state=0)]

In [ ]:
# # Seasonごとにcross validation
# for model in models:
#     scores = []
#     for season in seasons:
#         print('{}-{}'.format(season, model))
#         y_pred = cross_val_predict(model, X_train.query('Season==@season')[features], y_train[X_train['Season']==season],
#                                        cv=KFold(n_splits=5, shuffle=True, random_state=0), method='predict_proba')[:, 1]
#         score = log_loss(y_train[X_train['Season']==season], y_pred)
#         print(score)
#         scores.append(score)
#         print()
        
#     model.fit(X_train[features], y_train)
#     y_pred = model.predict_proba(MSampleSubmissionStage1Double[features])[:, 1]
#     MSampleSubmissionStage1Double['Pred'] = y_pred
#     MSampleSubmissionStage1Double[['ID', 'Pred']][:len(MSampleSubmissionStage1)].to_csv("{}-{}-{}.csv".format(int(time.time()), np.mean(scores), str(model)), index=False)
#     print("{}-cv_score:{}".format(model, np.mean(scores)))
#     print("{}-{}-{}.csv saved".format(int(time.time()), np.mean(scores), str(model)))
#     print()

In [ ]:
# そのシーズン以外のデータを使って，そのシーズンを予測．
y_preds = []
mean_scores = []
for model in models:
    scores = []
    y_pred_sub = []
    for season in seasons:
        print('{}-{}'.format(season, model))
        model.fit(X_train.query('Season!=@season')[features], y_train[X_train['Season']!=season])
        y_pred = model.predict_proba(X_train.query('Season==@season')[features])
        score = log_loss(y_train[X_train['Season']==season], y_pred)
        print(score)
        scores.append(score)
        print()
        y_pred_sub.append(model.predict_proba(MSampleSubmissionStage1Double[MSampleSubmissionStage1Double['Season']==season][features]))
    
    y_pred = np.concatenate(y_pred_sub)[:, 1]
    y_preds.append(y_pred)
    mean_scores.append(np.mean(scores))
    MSampleSubmissionStage1Double['Pred'] = y_pred
    MSampleSubmissionStage1Double[['ID', 'Pred']][:len(MSampleSubmissionStage1)].to_csv("{}-{}-{}.csv".format(int(time.time()), np.mean(scores), str(model)), index=False)
    print("{}-cv_score:{}".format(model, np.mean(scores)))
    print("{}-{}-{}.csv saved".format(int(time.time()), np.mean(scores), str(model)))
    print()

In [ ]:
y_pred_best = y_preds[mean_scores.index(min(mean_scores))]
y_pred_best
MSampleSubmissionStage1Double['Pred'] = y_pred
MSampleSubmissionStage1Double[['ID', 'Pred']][:len(MSampleSubmissionStage1)].to_csv("submission.csv", index=False)

In [ ]:
# for model in models:
#     print('{}'.format(model))
#     y_pred = cross_val_predict(model, X_train, y_train,
#                                     cv=KFold(n_splits=5, shuffle=True, random_state=0), method='predict_proba')[:, 1]
#     score = log_loss(y_train, y_pred)
#     print(score)
        
#     model.fit(X_train[features], y_train)
#     y_pred = model.predict_proba(MSampleSubmissionStage1Double[features])[:, 1]
#     MSampleSubmissionStage1Double['Pred'] = y_pred
#     MSampleSubmissionStage1Double[['ID', 'Pred']][:len(MSampleSubmissionStage1)].to_csv("{}-{}-{}.csv".format(int(time.time()), np.mean(scores), str(model)), index=False)
#     print("{}-cv_score:{}".format(model, np.mean(scores)))
#     print("{}-{}-{}.csv saved".format(int(time.time()), np.mean(scores), str(model)))
#     print()